<a href="https://colab.research.google.com/github/nobottle/deeplearning_pratice/blob/main/04_%EC%8B%A0%EA%B2%BD%EB%A7%9D%ED%95%99%EC%8A%B5_%EC%88%98%EC%B9%98%EB%AF%B8%EB%B6%84%EC%9D%84_%ED%86%B5%ED%95%9C_%ED%95%99%EC%8A%B5_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from common.functions import *
from common.gradient import numerical_gradient


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01): #input>입력층 뉴런개수, Hidden>은닉층 뉴런개수 output
        #>>출력층 뉴런개수 weight >> 가중치의 초기 표준편차 0.01
        # 가중치 초기화
        self.params = {} #셀프자리에 자식들이 들어옴 ex)net1=TwoLayerNet(3,3,3) ne1이 들어옴
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)#표준편차의 행렬을 만듬 그리고 위에 표준편차를 곱함
        self.params['b1'] = np.zeros(hidden_size)#제로벡터를 만듬 사이즈는 은닉층만큼
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)#표준정규분포를 따라 골라 표준편차를 곱함
        self.params['b2'] = np.zeros(output_size)#편향벡타는 제로벡터

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t): #t인 라벨과 계산한 확률이 얼마나 가까운지 엔트로피로 재겟다
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1) #axis=1행을 따라 최대가 일어나는 지점의 인덱스 값argmax
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy #정답률을 리턴해라 
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads


In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train),  (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True) #nomalize는 서로다른 것들 픽셀같은게 너무 다르면 이상하게 bias를 잡으므로 이걸함으로써 평탄화시킨다(?)
 
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼파라미터
iters_num = 10000  # 학습을 만번 한다
train_size = x_train.shape[0]#60000을 의미 (60000,784)에서
batch_size = 100   # 미니배치 크기
learning_rate = 0.1

train_loss_list = [] #손실함수 값들의 리스트 고도가 내려가는거를 저장
train_acc_list = []#몇문제 맞췃는지
test_acc_list = []#최종적 정확도

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1) #에퍽이란 데이터를 다 소진할떄까지 학습 600번학습을 하면 소진이 될거다

for i in range(iters_num):#9999까지 취하게 됨
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size) #랜덤하게 선택을 한다 60000중에 100개를
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0: #데이터600을 할때마다
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()
